In [ ]:
import geopandas as gpd
import requests
import json
from shapely.geometry import Polygon
from geojson import Feature, FeatureCollection
import time
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [ ]:
def up_load_shp(path):
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [ ]:
shapefile_path='../ags-iplan-data-fetcher-python/polygon_input/polygon.shp'
polygon_input=up_load_shp(shapefile_path)

In [ ]:
minx = polygon_input.bounds.minx[0]
miny = polygon_input.bounds.miny[0]
maxx = polygon_input.bounds.maxx[0]
maxy = polygon_input.bounds.maxy[0]

In [ ]:
url = r'https://ags.iplan.gov.il/arcgisiplan/rest/services/PlanningPublic/Xplan/MapServer/1/query?f=json&where=pl_area_dunam%20<%3D15&returnGeometry=true&geometry=%7B%22xmin%22%3A{}%2C%22ymin%22%3A{}%2C%22xmax%22%3A{}%2C%22ymax%22%3A{}%2C%22spatialReference%22%3A%7B%22wkid%22%3A2039%7D%7D&geometryType=esriGeometryEnvelope&inSR=2039&outFields=pl_number%2Cpl_name%2Cpl_url%2Cquantity_delta_120%2Cstation_desc%2Cplan_county_name&orderByFields=pl_number&outSR=2039'.format(minx, miny, maxx, maxy)
response = requests.get(url)
response_json = response.json()

In [ ]:
response_data = json.dumps(response_json)
dict = json.loads(response_data)

xplan_plans_data = dict['features']
# xplan_plans_data
xplan_plans_data_filtered = []

for plan in xplan_plans_data:
    interior_rings = plan['geometry']['rings'][1:]
    exterior_ring=plan['geometry']['rings'][0]
    polygon = Polygon(exterior_ring, holes=interior_rings)
    if polygon_input.contains(polygon.centroid).bool():
        xplan_plans_data_filtered.append(plan)

In [ ]:
# chrome_options = webdriver.ChromeOptions()

# prefs = {"profile.default_content_setting_values.notifications" : 2}

# chrome_options.add_experimental_option("prefs",prefs)

PATH = 'chromedriver-win64/chromedriver.exe'

service = Service(executable_path=PATH)

mydivs_list = []

new_plans_data = []

browser = Browser('chrome', headless=True)

for plan in xplan_plans_data_filtered:
    lis_elements = []
    browser.visit(plan['attributes']['pl_url'])
    time.sleep(7)
    buttons = browser.find_by_tag('button')
    for button in buttons:
        if button.text == 'נתונים נוספים':
            button.click()
            
    soup = BeautifulSoup(browser.html, 'html.parser')
    mydivs = soup.find_all("li", {"class": "sv4-icon-arrow uk-open uk-hide-arrow ng-star-inserted"})
    mydivs_list.append(mydivs)

    for divs in mydivs:
        div = divs.find_all('div', attrs={'class': 'uk-accordion-content uk-margin-remove'})

    results = div[0].find_all('ul')

    for result in results:
        li = result.find_all('li')
        lis_elements.append(li)
        
    obj = {}
    for element in lis_elements:
        quantitative_data_main_header = element[0].find_all('div', {'class': 'uk-width-expand ng-star-inserted'})[0].text.strip()
        quantitative_data_main_value = element[0].find_all('div', {'class': 'uk-width-1-2 uk-text-left'})[0].find_all('b')[0].text
        obj[quantitative_data_main_header] = quantitative_data_main_value

        plan['attributes'].update(obj)    
    print(plan)
browser.quit()

In [ ]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
features = []

key_mapping = {
    'חדרי מלון / תיירות (חדר)': 'hotel_rooms_room',
    'חדרי מלון / תיירות (מ"ר)': 'hotel_rooms_square_meters',
    'מבני ציבור (מ"ר)': 'public_buildings_square_meters',
    'מגורים (יח"ד)': 'residence_housing_units',
    'מגורים (מ"ר)': 'residence_square_meters',
    'מסחר (מ"ר)': 'trade_square_meters',
    'תעסוקה (מ"ר)': 'employment_square_meters',
    'דירות קטנות (יח"ד)': 'small_apartments_units',
    'דירות להשכרה (יח"ד)': 'apartments_for_rent_units'
}

for plan in xplan_plans_data:
        # Create a new dictionary to store the updated attributes
    new_attributes = {}
    # Loop over the attributes dictionary in each item
    for key, value in plan['attributes'].items():
        # Check if the key needs to be replaced
        if key in key_mapping:
            # Replace the key
            new_key = key_mapping[key]
            # Add the new key with the same value
            new_attributes[new_key] = value
        else:
            # If key doesn't need to be replaced, keep it as it is
            new_attributes[key] = value
    # Update the attributes with the new dictionary
    plan['attributes'] = new_attributes

    interior_rings = plan['geometry']['rings'][1:]
    exterior_ring=plan['geometry']['rings'][0]
    polygon = Polygon(exterior_ring, holes=interior_rings)
    if polygon_input.contains(polygon.centroid).bool():
        feature = Feature(geometry=polygon, properties=plan['attributes'])
        features.append(feature)
        
crs = {
    "type": "name",
    "properties": {
        "name": "EPSG:2039"
    }
}
        
feature_collection = FeatureCollection(features)

gdf = gpd.GeoDataFrame.from_features(feature_collection, crs='EPSG:2039')
gdf.to_file(r'my_polygon/{}_polygon.shp'.format(file_date), encoding='utf-8')
gdf